# Radni tijek s ljudskom interakcijom uz Microsoft Agent Framework

## 🎯 Ciljevi učenja

U ovom bilježniku naučit ćete kako implementirati radne tijekove s **ljudskom interakcijom** koristeći `RequestInfoExecutor` iz Microsoft Agent Frameworka. Ovaj moćan obrazac omogućuje pauziranje AI radnih tijekova kako bi se prikupio ljudski unos, čineći vaše agente interaktivnima i dajući ljudima kontrolu nad ključnim odlukama.

## 🔄 Što je radni tijek s ljudskom interakcijom?

**Radni tijek s ljudskom interakcijom (HITL)** je dizajnerski obrazac gdje AI agenti pauziraju izvršenje kako bi zatražili ljudski unos prije nastavka. Ovo je ključno za:

- ✅ **Važne odluke** - Dobivanje ljudskog odobrenja prije poduzimanja važnih radnji
- ✅ **Nejasne situacije** - Omogućavanje ljudima da razjasne kada AI nije siguran
- ✅ **Korisničke preferencije** - Pitanje korisnika da odaberu između više opcija
- ✅ **Usklađenost i sigurnost** - Osiguravanje ljudskog nadzora za regulirane operacije
- ✅ **Interaktivna iskustva** - Izgradnja razgovornih agenata koji odgovaraju na korisnički unos

## 🏗️ Kako funkcionira u Microsoft Agent Frameworku

Framework pruža tri ključne komponente za HITL:

1. **`RequestInfoExecutor`** - Poseban izvršitelj koji pauzira radni tijek i emitira `RequestInfoEvent`
2. **`RequestInfoMessage`** - Osnovna klasa za tipizirane zahtjeve koji se šalju ljudima
3. **`RequestResponse`** - Povezuje ljudske odgovore s izvornim zahtjevima koristeći `request_id`

**Obrazac radnog tijeka:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Naš primjer: Rezervacija hotela uz korisničku potvrdu

Nadogradit ćemo uvjetni radni tijek dodavanjem ljudske potvrde **prije** predlaganja alternativnih destinacija:

1. Korisnik zatraži destinaciju (npr. "Pariz")
2. `availability_agent` provjerava dostupnost soba
3. **Ako nema soba** → `confirmation_agent` pita "Želite li vidjeti alternative?"
4. Radni tijek **pauzira** koristeći `RequestInfoExecutor`
5. **Čovjek odgovara** "da" ili "ne" putem unosa u konzolu
6. `decision_manager` usmjerava na temelju odgovora:
   - **Da** → Prikazuje alternativne destinacije
   - **Ne** → Otkazuje zahtjev za rezervaciju
7. Prikazuje konačni rezultat

Ovo pokazuje kako korisnicima dati kontrolu nad prijedlozima agenta!

---

Krenimo! 🚀


## Korak 1: Uvoz potrebnih knjižnica

Uvozimo standardne komponente Agent Frameworka plus **klase specifične za ljudsku interakciju**:
- `RequestInfoExecutor` - Izvršitelj koji zaustavlja tijek rada radi unosa od strane čovjeka
- `RequestInfoEvent` - Događaj koji se emitira kada se traži unos od strane čovjeka
- `RequestInfoMessage` - Osnovna klasa za tipizirane podatke zahtjeva
- `RequestResponse` - Povezuje odgovore ljudi sa zahtjevima
- `WorkflowOutputEvent` - Događaj za otkrivanje izlaza tijeka rada


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Korak 2: Definiranje Pydantic modela za strukturirane izlaze

Ovi modeli definiraju **sheme** koje će agenti vraćati. Zadržavamo sve modele iz uvjetnog tijeka rada i dodajemo:

**Novo za ljudsku intervenciju:**
- `HumanFeedbackRequest` - Podklasa `RequestInfoMessage` koja definira podatke zahtjeva poslane ljudima
  - Sadrži `prompt` (pitanje koje treba postaviti) i `destination` (kontekst o nedostupnom gradu)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Korak 3: Izradite alat za rezervaciju hotela

Isti alat iz uvjetnog tijeka rada - provjerava dostupnost soba na odredištu.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Korak 4: Definiranje funkcija uvjeta za usmjeravanje

Potrebne su nam **četiri funkcije uvjeta** za naš radni tijek s ljudskom intervencijom:

**Iz uvjetnog radnog tijeka:**
1. `has_availability_condition` - Usmjerava kada hoteli JESU dostupni
2. `no_availability_condition` - Usmjerava kada hoteli NISU dostupni

**Novo za radni tijek s ljudskom intervencijom:**
3. `user_wants_alternatives_condition` - Usmjerava kada korisnik kaže "da" na alternative
4. `user_declines_alternatives_condition` - Usmjerava kada korisnik kaže "ne" na alternative


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Korak 5: Kreirajte Decision Manager Executor

Ovo je **srž uzorka s ljudskom intervencijom**! `DecisionManager` je prilagođeni `Executor` koji:

1. **Prima povratne informacije od ljudi** putem objekata `RequestResponse`
2. **Obrađuje korisnikovu odluku** (da/ne)
3. **Usmjerava tijek rada** slanjem poruka odgovarajućim agentima

Ključne značajke:
- Koristi dekorator `@handler` za izlaganje metoda kao koraka tijeka rada
- Prima `RequestResponse[HumanFeedbackRequest, str]` koji sadrži i izvorni zahtjev i korisnikov odgovor
- Generira jednostavne poruke "da" ili "ne" koje pokreću naše funkcije uvjeta


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Korak 6: Kreirajte prilagođeni izvršitelj prikaza

Isti izvršitelj prikaza iz uvjetnog tijeka rada - daje konačne rezultate kao izlaz tijeka rada.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Korak 7: Učitajte varijable okruženja

Konfigurirajte LLM klijent (GitHub Models, Azure OpenAI ili OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Korak 8: Kreirajte AI agente i izvršitelje

Kreiramo **šest komponenti tijeka rada**:

**Agenti (omotani u AgentExecutor):**
1. **availability_agent** - Provjerava dostupnost hotela pomoću alata
2. **confirmation_agent** - 🆕 Priprema zahtjev za potvrdu od strane čovjeka
3. **alternative_agent** - Predlaže alternativne gradove (kada korisnik kaže da)
4. **booking_agent** - Potiče rezervaciju (kada su sobe dostupne)
5. **cancellation_agent** - 🆕 Rukuje porukom o otkazivanju (kada korisnik kaže ne)

**Posebni izvršitelji:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` koji pauzira tijek rada za unos od strane čovjeka
7. **decision_manager** - 🆕 Prilagođeni izvršitelj koji usmjerava na temelju odgovora čovjeka (već definiran gore)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Korak 9: Izgradnja tijeka rada s ljudskom intervencijom

Sada konstruiramo graf tijeka rada s **uvjetnim usmjeravanjem**, uključujući putanju s ljudskom intervencijom:

**Struktura tijeka rada:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Ključne veze:**
- `availability_agent → confirmation_agent` (kada nema soba)
- `confirmation_agent → prepare_human_request` (transformacija tipa)
- `prepare_human_request → request_info_executor` (pauza za ljudsku intervenciju)
- `request_info_executor → decision_manager` (uvijek - pruža RequestResponse)
- `decision_manager → alternative_agent` (kada korisnik kaže "da")
- `decision_manager → cancellation_agent` (kada korisnik kaže "ne")
- `availability_agent → booking_agent` (kada su sobe dostupne)
- Svi putevi završavaju na `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Korak 10: Pokrenite Testni Slučaj 1 - Grad BEZ Dostupnosti (Pariz s Ljudskom Potvrdom)

Ovaj test prikazuje **cijeli ciklus s ljudskom intervencijom**:

1. Zatražite hotel u Parizu
2. availability_agent provjerava → Nema soba
3. confirmation_agent stvara pitanje za ljudsku interakciju
4. request_info_executor **pauzira tijek rada** i emitira `RequestInfoEvent`
5. **Aplikacija detektira događaj i traži unos korisnika u konzoli**
6. Korisnik upisuje "da" ili "ne"
7. Aplikacija šalje odgovor putem `send_responses_streaming()`
8. decision_manager usmjerava na temelju odgovora
9. Prikazuje se konačni rezultat

**Ključni Uzorak:**
- Koristite `workflow.run_stream()` za prvu iteraciju
- Koristite `workflow.send_responses_streaming(pending_responses)` za sljedeće iteracije
- Slušajte `RequestInfoEvent` kako biste detektirali kada je potrebna ljudska interakcija
- Slušajte `WorkflowOutputEvent` za hvatanje konačnih rezultata


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Korak 11: Pokreni Testni Slučaj 2 - Grad S Dostupnošću (Stockholm - Nije Potreban Ljudski Unos)

Ovaj test prikazuje **direktni put** kada su sobe dostupne:

1. Zatraži hotel u Stockholmu
2. availability_agent provjerava → Sobe dostupne ✅
3. booking_agent predlaže rezervaciju
4. display_result prikazuje potvrdu
5. **Nije potreban ljudski unos!**

Tijek rada potpuno zaobilazi put s ljudskim sudjelovanjem kada su sobe dostupne.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Ključne točke i najbolje prakse za ljudsku interakciju

### ✅ Što ste naučili:

#### 1. **RequestInfoExecutor obrazac**
Obrazac za ljudsku interakciju u Microsoft Agent Frameworku koristi tri ključne komponente:
- `RequestInfoExecutor` - Zaustavlja tijek rada i emitira događaje
- `RequestInfoMessage` - Osnovna klasa za tipizirane zahtjeve (naslijedite ovu klasu!)
- `RequestResponse` - Povezuje ljudske odgovore s izvornim zahtjevima

**Ključna razumijevanja:**
- `RequestInfoExecutor` NE prikuplja unos sam - samo zaustavlja tijek rada
- Vaš aplikacijski kod mora slušati `RequestInfoEvent` i prikupljati unos
- Morate pozvati `send_responses_streaming()` s rječnikom koji mapira `request_id` na korisnikov odgovor

#### 2. **Obrazac za streaming izvršenje**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Arhitektura temeljena na događajima**
Slušajte specifične događaje za kontrolu tijeka rada:
- `RequestInfoEvent` - Potreban je ljudski unos (tijek rada zaustavljen)
- `WorkflowOutputEvent` - Dostupan je konačni rezultat (tijek rada završen)
- `WorkflowStatusEvent` - Promjene stanja (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, itd.)

#### 4. **Prilagođeni izvršitelji s @handler**
`DecisionManager` pokazuje kako stvoriti izvršitelje koji:
- Koriste dekorator `@handler` za izlaganje metoda kao koraka tijeka rada
- Primaju tipizirane poruke (npr. `RequestResponse[HumanFeedbackRequest, str]`)
- Usmjeravaju tijek rada slanjem poruka drugim izvršiteljima
- Pristupaju kontekstu putem `WorkflowContext`

#### 5. **Uvjetno usmjeravanje s ljudskim odlukama**
Možete stvoriti funkcije uvjeta koje procjenjuju ljudske odgovore:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Primjene u stvarnom svijetu:

1. **Radni tijekovi odobrenja**
   - Dobivanje odobrenja menadžera prije obrade izvještaja o troškovima
   - Zahtijevanje ljudske provjere prije slanja automatiziranih e-mailova
   - Potvrda transakcija visoke vrijednosti prije izvršenja

2. **Moderacija sadržaja**
   - Označavanje sumnjivog sadržaja za ljudsku provjeru
   - Traženje od moderatora da donesu konačnu odluku u graničnim slučajevima
   - Eskalacija prema ljudima kada je AI sigurnost niska

3. **Korisnička podrška**
   - Dopustite AI-u da automatski rješava rutinska pitanja
   - Eskalirajte složena pitanja ljudskim agentima
   - Pitajte korisnika želi li razgovarati s čovjekom

4. **Obrada podataka**
   - Traženje od ljudi da riješe nejasne unose podataka
   - Potvrda AI interpretacija nejasnih dokumenata
   - Dopustite korisnicima da biraju između više valjanih interpretacija

5. **Sustavi kritični za sigurnost**
   - Zahtijevanje ljudske potvrde prije nepovratnih radnji
   - Dobivanje odobrenja prije pristupa osjetljivim podacima
   - Potvrda odluka u reguliranim industrijama (zdravstvo, financije)

6. **Interaktivni agenti**
   - Izgradnja razgovornih botova koji postavljaju dodatna pitanja
   - Stvaranje čarobnjaka koji vode korisnike kroz složene procese
   - Dizajniranje agenata koji surađuju s ljudima korak po korak

### 🔄 Usporedba: S i bez ljudske interakcije

| Značajka | Uvjetni tijek rada | Tijek rada s ljudskom interakcijom |
|----------|---------------------|------------------------------------|
| **Izvršenje** | Jedan `workflow.run()` | Petlja s `run_stream()` + `send_responses_streaming()` |
| **Korisnički unos** | Nema (potpuno automatizirano) | Interaktivni upiti putem `input()` ili UI |
| **Komponente** | Agenti + izvršitelji | + RequestInfoExecutor + DecisionManager |
| **Događaji** | Samo AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, itd. |
| **Zaustavljanje** | Nema zaustavljanja | Tijek rada se zaustavlja na RequestInfoExecutor |
| **Ljudska kontrola** | Nema ljudske kontrole | Ljudi donose ključne odluke |
| **Primjena** | Automatizacija | Suradnja i nadzor |

### 🚀 Napredni obrasci:

#### Više točaka za ljudske odluke
Možete imati više `RequestInfoExecutor` čvorova u istom tijeku rada:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Rukovanje vremenskim ograničenjima
Implementirajte vremenska ograničenja za ljudske odgovore:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integracija bogatog UI-a
Umjesto `input()`, integrirajte s web UI-jem, Slackom, Teamsom, itd.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Uvjetna ljudska interakcija
Tražite ljudski unos samo u specifičnim situacijama:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Najbolje prakse:

1. **Uvijek naslijedite RequestInfoMessage**
   - Osigurava sigurnost tipova i validaciju
   - Omogućuje bogat kontekst za prikaz u UI-u
   - Pojašnjava namjeru svake vrste zahtjeva

2. **Koristite opisne upite**
   - Uključite kontekst o tome što tražite
   - Objasnite posljedice svakog izbora
   - Držite pitanja jednostavnim i jasnim

3. **Rukovanje neočekivanim unosom**
   - Validirajte korisničke odgovore
   - Osigurajte zadane vrijednosti za nevažeći unos
   - Dajte jasne poruke o greškama

4. **Praćenje ID-ova zahtjeva**
   - Koristite korelaciju između request_id i odgovora
   - Nemojte pokušavati ručno upravljati stanjem

5. **Dizajnirajte za neblokiranje**
   - Nemojte blokirati niti čekajući unos
   - Koristite asinkrone obrasce
   - Podržite istovremene instance tijeka rada

### 📚 Povezani koncepti:

- **Agent Middleware** - Presretanje poziva agenata (prethodni bilježnik)
- **Upravljanje stanjem tijeka rada** - Pohrana stanja tijeka rada između pokretanja
- **Suradnja više agenata** - Kombiniranje ljudske interakcije s timovima agenata
- **Arhitekture temeljene na događajima** - Izgradnja reaktivnih sustava s događajima

---

### 🎓 Čestitamo!

Savladali ste tijekove rada s ljudskom interakcijom u Microsoft Agent Frameworku! Sada znate kako:
- ✅ Zaustaviti tijekove rada za prikupljanje ljudskog unosa
- ✅ Koristiti RequestInfoExecutor i RequestInfoMessage
- ✅ Rukovati streaming izvršenjem s događajima
- ✅ Stvoriti prilagođene izvršitelje s @handler
- ✅ Usmjeravati tijekove rada na temelju ljudskih odluka
- ✅ Izgraditi interaktivne AI agente koji surađuju s ljudima

**Ovo je ključni obrazac za izgradnju pouzdanih, kontrolabilnih AI sustava!** 🚀



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne odgovaramo za nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
